In [15]:
import requests
import json

apiKey = 'a8b3e6d4eaee404ba7b32cd80110a2eb'
url = 'https://script.google.com/a/upi.edu/macros/s/AKfycbwqznIpOsEC_cPAe4pTy4pEQBN5MaksSaIy0IL8kganMDDeDYchgsebQb1yeEoSpxet/exec?api=sampah&apiKey='
url += apiKey

response = requests.get(url)
data = response.json()

print(json.dumps(data, indent=4))

{
    "data": [
        {
            "date": "17-01-2024",
            "name": "customer-1",
            "wasteType": "Dry Box",
            "weight": 5.1,
            "price": 4079.9999999999995
        },
        {
            "date": "17-01-2024",
            "name": "customer-1",
            "wasteType": "Mixed Bucket",
            "weight": 2,
            "price": 1000
        },
        {
            "date": "17-01-2024",
            "name": "customer-1",
            "wasteType": "Aqua Clean Bottle",
            "weight": 0.8,
            "price": 1600
        },
        {
            "date": "17-01-2024",
            "name": "customer-1",
            "wasteType": "Crackle",
            "weight": 1.6,
            "price": 640
        },
        {
            "date": "17-01-2024",
            "name": "customer-1",
            "wasteType": "Dry Duplex",
            "weight": 1,
            "price": 400
        },
        {
            "date": "17-01-2024",
            "name": "cus

In [16]:
import pandas as pd

df = pd.DataFrame(data['data'])

display(df)

,date,name,wasteType,weight,price
0,17-01-2024,customer-1,Dry Box,5.1,4080.0
1,17-01-2024,customer-1,Mixed Bucket,2.0,1000.0
2,17-01-2024,customer-1,Aqua Clean Bottle,0.8,1600.0
3,17-01-2024,customer-1,Crackle,1.6,640.0
4,17-01-2024,customer-1,Dry Duplex,1.0,400.0
...,...,...,...,...,...
461,12-06-2024,customer-30,Mixed Bucket,0.6,300.0
462,12-06-2024,customer-30,Mixed Bucket,0.5,250.0
463,12-06-2024,customer-30,LD/gallon cap,0.1,200.0
464,12-06-2024,customer-30,Mixed Bucket,0.1,50.0


In [17]:
import plotly.express as px
import plotly.graph_objs as go

"""
    Total Revenue and Total Waste
    Scorecard Text
"""
total_revenue = df['price'].sum()
total_waste = df['weight'].sum()


""" 
    Top 5 Waste Type
    Horizontal Bar Chart
"""
wasteType_sum = df.groupby('wasteType')['weight'].count().reset_index()
wasteType_sum_sort = wasteType_sum.sort_values(by='weight', ascending=False)
wasteType_sum = wasteType_sum_sort.sort_values(by='weight', ascending=False).head(5)
fig_wasteType = px.bar(wasteType_sum, x='weight', y='wasteType', orientation='h', title='Type of Waste')


""" 
    4 Newest Date
    Line Chart
"""
df['date'] = pd.to_datetime(df['date'], format='%d-%m-%Y')
waste_trend = df.groupby('date')['weight'].count().reset_index()
newest_waste_trend = waste_trend.sort_values(by='date', ascending=False)
newest_waste_trend = newest_waste_trend.sort_values(by='date', ascending=False)
fig_waste_trend = px.line(newest_waste_trend, x='date', y='weight', orientation='h', title='Waste Trend')


""" 
    Most Consistent Customers
    Heat Map
"""
consistent_customers = df['name'].value_counts().reset_index()
fig_consistent_customers = px.treemap(consistent_customers, path=[consistent_customers.name], values='count', title='Most Consistent Customers')


""" 
    Target Customer
    Gauge Chart
"""
target_customer_count = df['name'].nunique()
max_target_customer = 50  # Set the maximum target customer count here
fig_target_customer = go.Figure(go.Indicator(
    mode="gauge+number",
    value=target_customer_count,
    domain={'x': [0, 1], 'y': [0, 1]},
    title={'text': "Target Customer"},
    gauge={'axis': {'range': [None, max_target_customer]},
           'bar': {'color': "darkblue"},
           'steps': [
               {'range': [0, max_target_customer / 2], 'color': "lightgray"}],
           'threshold': {'line': {'color': "red", 'width': 4}, 'thickness': 0.75, 'value': target_customer_count}}))
fig_target_customer.update_layout()


""" 
    Customer Development
    Vertical Bar Chart
"""
customer_development = df.groupby('date')['name'].nunique().reset_index()
customer_development_trend = customer_development.sort_values(by='date', ascending=False).head(4)
customer_development_trend = customer_development_trend.sort_values(by='date', ascending=False)
fig_customer_development = px.bar(customer_development_trend, x='date', y='name', title='Customer Development')


In [18]:
# Install the required libraries
# !pip install dash jupyter_dash

from dash import Dash, dcc, html
from jupyter_dash import JupyterDash

# Create the Dash app layout
app = JupyterDash(__name__, external_stylesheets=['https://cdn.jsdelivr.net/npm/bootstrap@5.3.3/dist/css/bootstrap.min.css'])
app.layout = html.Div([
    html.Div([
        html.H1('Waste Bank Dashboard', className='text-center my-5'),
        html.Div([
            html.Div([
                html.Div([
                    html.P('Total Revenue', className='h3'),
                    html.P(f'Rp {total_revenue}', className='h1'),
                ]),
                html.Div([
                    html.P('Total Waste', className='h3'),
                    html.P(f'{total_waste} kg', className='h1'),
                ], className='mt-5'),
            ], className='col-lg-4 card'),
            html.Div([
                dcc.Graph(figure=fig_wasteType),
            ], className='col-lg-4 card'),
            html.Div([
                dcc.Graph(figure=fig_waste_trend),
            ], className='col-lg-4 card'),
        ], className='row align-items-center'),
        html.Div([
            html.Div([
                dcc.Graph(figure=fig_consistent_customers),
            ], className='col-lg-4 card'),
            html.Div([
                dcc.Graph(figure=fig_target_customer),
            ], className='col-lg-4 card'),
            html.Div([
                dcc.Graph(figure=fig_customer_development),
            ], className='col-lg-4 card'),
        ], className='row align-items-center'),
    ], className="container")
])

# Run the app
app.run_server(mode='inline')

C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\dash\dash.py:556: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.

